In [16]:
!pip install pycaret
!git clone https://github.com/alikolling/clustering_audi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'clustering_audi'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 2), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), 19.47 KiB | 1.39 MiB/s, done.
Filtering content: 100% (3/3), 419.94 MiB | 48.01 MiB/s, done.


In [1]:
import pandas as pd
import numpy as np
#from pycaret.regression import *
from pycaret.clustering import *
import warnings
#import seaborn as sns
#import matplotlib.pyplot as plt

import os
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
from pycaret.utils import version
version()

'3.0.0'

In [3]:
%cd clustering_audi/


/content/clustering_audi


In [4]:
!ls

can_bus_audi	 Clustering_CANbus_Audi.ipynb  README.md
clustering_audi  logs.log


In [5]:
data = []

for diri in os.listdir('can_bus_audi/'):
    for i in os.listdir('can_bus_audi/'+diri+'/bus'):
        data.append(pd.read_json('can_bus_audi/'+diri+'/bus/'+i))
        #with open('can_bus_audi/'+diri+'/bus/'+i,'r') as f:
            #print('can_bus_audi/'+diri+'/bus/'+i)
            #pd.([data, pd.json_normalize(json.loads(f.read()))])
            
df_data = pd.concat(data).drop('unit')

In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, values to values
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   acceleration_x                   3 non-null      object
 1   acceleration_y                   3 non-null      object
 2   acceleration_z                   3 non-null      object
 3   accelerator_pedal                3 non-null      object
 4   accelerator_pedal_gradient_sign  3 non-null      object
 5   angular_velocity_omega_x         3 non-null      object
 6   angular_velocity_omega_y         3 non-null      object
 7   angular_velocity_omega_z         3 non-null      object
 8   brake_pressure                   3 non-null      object
 9   distance_pulse_front_left        3 non-null      object
 10  distance_pulse_front_right       3 non-null      object
 11  distance_pulse_rear_left         3 non-null      object
 12  distance_pulse_rear_right        3 

In [7]:
df_data.isna().sum()

acceleration_x                     0
acceleration_y                     0
acceleration_z                     0
accelerator_pedal                  0
accelerator_pedal_gradient_sign    0
angular_velocity_omega_x           0
angular_velocity_omega_y           0
angular_velocity_omega_z           0
brake_pressure                     0
distance_pulse_front_left          0
distance_pulse_front_right         0
distance_pulse_rear_left           0
distance_pulse_rear_right          0
latitude_degree                    0
latitude_direction                 0
longitude_degree                   0
longitude_direction                0
pitch_angle                        0
roll_angle                         0
steering_angle_calculated          0
steering_angle_calculated_sign     0
vehicle_speed                      0
dtype: int64

In [8]:
df_data.head()

,acceleration_x,acceleration_y,acceleration_z,accelerator_pedal,accelerator_pedal_gradient_sign,angular_velocity_omega_x,angular_velocity_omega_y,angular_velocity_omega_z,brake_pressure,distance_pulse_front_left,...,distance_pulse_rear_right,latitude_degree,latitude_direction,longitude_degree,longitude_direction,pitch_angle,roll_angle,steering_angle_calculated,steering_angle_calculated_sign,vehicle_speed
values,"[[1533906414544846, 4.319999999999936], [15339...","[[1533906414544846, 0.419999999999959], [15339...","[[1533906414544858, 9.580000000000041], [15339...","[[1533906414544966, 38.8], [1533906414554935, ...","[[1533906414544966, 0], [1533906414554935, 0],...","[[1533906414544858, -0.550000000000011], [1533...","[[1533906414544858, -1.009999999999991], [1533...","[[1533906414544846, 3.370000000000004], [15339...","[[1533906414549891, 0.0], [1533906414559893, 0...","[[1533906414554882, 179], [1533906414574879, 1...",...,"[[1533906414554882, 464], [1533906414574879, 4...","[[1533906414558865, 48.788047], [1533906414718...","[[1533906414558865, 0], [1533906414718868, 0],...","[[1533906414558865, 11.382514], [1533906414718...","[[1533906414558865, 0], [1533906414718868, 0],...","[[1533906414544858, -1.075000000000002], [1533...","[[1533906414544858, 1.700000000000002], [15339...","[[1533906414543862, 52.35], [1533906414553856,...","[[1533906414543862, 0], [1533906414553856, 0],...","[[1533906414554882, 10.36], [1533906414574879,..."
values,"[[1554115464693201, -0.5], [1554115464698160, ...","[[1554115464693201, 0.319999999999936], [15541...","[[1554115464693212, 9.819999999999936], [15541...","[[1554115464699170, 0.0], [1554115464709186, 0...","[[1554115464699170, 0], [1554115464709186, 0],...","[[1554115464693212, 0.39999999999997704], [155...","[[1554115464693212, -0.519999999999981], [1554...","[[1554115464693201, -0.13999999999998602], [15...","[[1554115464694125, 6.299999999999997], [15541...","[[1554115464699122, 88], [1554115464719125, 89...",...,"[[1554115464699122, 546], [1554115464719125, 5...","[[1554115464763134, 48.142837], [1554115464923...","[[1554115464763134, 0], [1554115464923127, 0],...","[[1554115464763134, 11.576722], [1554115464923...","[[1554115464763134, 0], [1554115464923127, 0],...","[[1554115464699097, -0.125], [1554115464709106...","[[1554115464699097, 2.450000000000003], [15541...","[[1554115464698116, 2.4], [1554115464708106, 2...","[[1554115464698116, 1], [1554115464708106, 1],...","[[1554115464699122, 6.54], [1554115464719125, ..."
values,"[[1554121593489120, 0.319999999999936], [15541...","[[1554121593489120, 0.220000000000027], [15541...","[[1554121593489128, 10.0], [1554121593494102, ...","[[1554121593495095, 22.8], [1554121593505094, ...","[[1554121593495095, 0], [1554121593505094, 0],...","[[1554121593489128, -0.40000000000003405], [15...","[[1554121593489128, 0.49000000000000904], [155...","[[1554121593489120, -0.32999999999998403], [15...","[[1554121593490045, 0.0], [1554121593500040, 0...","[[1554121593505032, 391], [1554121593525035, 4...",...,"[[1554121593505032, 759], [1554121593525035, 7...","[[1554121593609081, 48.771678], [1554121593769...","[[1554121593609081, 0], [1554121593769081, 0],...","[[1554121593609081, 11.418433], [1554121593769...","[[1554121593609081, 0], [1554121593769081, 0],...","[[1554121593494102, 0.799999999999997], [15541...","[[1554121593494102, 1.025000000000005], [15541...","[[1554121593494041, 0.45], [1554121593504050, ...","[[1554121593494041, 1], [1554121593504050, 1],...","[[1554121593505032, 41.47], [1554121593525035,..."


In [9]:
clean_data = []
for i in df_data.columns:
    clean_data.append(np.asanyarray(df_data[i].values.all()).flatten().reshape(-1,2)[:,1])

In [10]:
df = pd.DataFrame(data=clean_data).transpose()

In [11]:
df = pd.DataFrame(data=df.values, columns=df_data.columns)

In [12]:
df.head()

,acceleration_x,acceleration_y,acceleration_z,accelerator_pedal,accelerator_pedal_gradient_sign,angular_velocity_omega_x,angular_velocity_omega_y,angular_velocity_omega_z,brake_pressure,distance_pulse_front_left,...,distance_pulse_rear_right,latitude_degree,latitude_direction,longitude_degree,longitude_direction,pitch_angle,roll_angle,steering_angle_calculated,steering_angle_calculated_sign,vehicle_speed
0,0.32,0.22,10.00,22.8,0.0,-0.40,0.49,-0.33,0.0,391.0,...,759.0,48.771678,0.0,11.418433,0.0,0.800,1.025,0.45,1.0,41.47
1,0.36,0.24,10.08,22.8,0.0,-0.48,0.52,-0.31,0.0,401.0,...,768.0,48.771678,0.0,11.418433,0.0,0.825,1.025,0.45,1.0,41.53
2,0.40,0.16,10.08,22.8,0.0,-0.67,0.55,-0.35,0.0,410.0,...,777.0,48.771678,0.0,11.418433,0.0,0.825,1.025,0.45,1.0,41.57
3,0.40,0.18,10.06,22.8,0.0,-0.46,0.55,-0.38,-0.3,420.0,...,787.0,48.771678,0.0,11.418433,0.0,0.825,1.025,0.45,1.0,41.58
4,0.36,0.24,10.06,22.8,0.0,-0.52,0.40,-0.44,0.0,429.0,...,796.0,48.771697,0.0,11.418584,0.0,0.825,1.000,0.45,1.0,41.64


In [17]:
df = df.dropna()

In [18]:
from sklearn.cluster import KMeans

k_means = KMeans(init="k-means++", n_clusters=3, n_init=10)

k_means.fit(df)

KMeans(n_clusters=3, n_init=10)

In [65]:
exp = setup(data=df,
            normalize=True,
            normalize_method='minmax',
            session_id=42,
            #ignore_features=['distance_pulse_front_left ','distance_pulse_front_right','distance_pulse_rear_left','distance_pulse_rear_right',
                             #'accelerator_pedal_gradient_sign', 'latitude_direction', 'longitude_direction', 
                             #'angular_velocity_omega_x', 'angular_velocity_omega_y', 'angular_velocity_omega_z'],
            remove_outliers=True
           )

,Description,Value
0,Session id,42
1,Original data shape,"(3726, 22)"
2,Transformed data shape,"(3539, 22)"
3,Numeric features,22
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,Remove outliers,True
9,Outliers threshold,0.050000


In [69]:
kmeans = create_model('kmeans', num_clusters=5)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.2841,938.9045,1.4408,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [71]:

plot_model(kmeans, plot='cluster')

In [72]:
a = assign_model(kmeans, transformation=True)

In [73]:
s = a.loc[a.Cluster == 'Cluster 1']

In [75]:
a.size

81397